SOAP FOR XYZ FILES

load and prepare

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from mordred import Calculator, descriptors
import os
import numpy, math, random
#from visualise import view
from ase import Atoms
# import sys
# sys.path.insert(0, './data/descriptor_codes/')
# sys.path.insert(0, './data/descriptor_codes/src')
from dscribe.descriptors import SOAP

In [2]:
with open('data/rop313/orop/1/1.b973c.xyz', 'r') as xyz_file:
    lines = xyz_file.readlines()[2:]
atomic_symbols = []
for line in lines:
    atomic_symbols.append(line.split()[0])
import numpy as np
 
atomic_coordinates = np.array([line.split()[1:4] for line in lines], dtype=float)
atomic_symbols1 = set(atomic_symbols)
atomic_symbols1 = list(atomic_symbols1)
print(atomic_symbols1)
print('\n')
print(atomic_coordinates)


['H', 'C']


[[ 2.43657686e+00 -6.92133254e-01  2.07812401e-05]
 [ 2.43658018e+00  6.92124014e-01  2.72198768e-05]
 [ 1.22901070e+00  1.39397634e+00  1.99468715e-05]
 [ 3.43342043e-06  7.12530909e-01  7.14065790e-06]
 [-1.22900043e+00  1.39398219e+00  1.00629271e-06]
 [-2.43657341e+00  6.92135711e-01 -1.17710472e-05]
 [-2.43657675e+00 -6.92121579e-01 -1.81013037e-05]
 [-1.22900727e+00 -1.39397382e+00 -1.19756425e-05]
 [ 3.65049300e-08 -7.12528466e-01  5.46229070e-07]
 [ 1.22900387e+00 -1.39397977e+00  7.29993441e-06]
 [ 1.23663547e+00 -2.47558383e+00  1.71706349e-06]
 [-1.23664378e+00 -2.47557790e+00 -1.66952523e-05]
 [-3.37040600e+00 -1.23506617e+00 -2.74312415e-05]
 [-3.37040002e+00  1.23508487e+00 -1.67141879e-05]
 [-1.23663196e+00  2.47558627e+00  6.51593895e-06]
 [ 1.23664735e+00  2.47558038e+00  2.35875126e-05]
 [ 3.37040941e+00  1.23506867e+00  3.75449178e-05]
 [ 3.37040340e+00 -1.23508256e+00  2.60821397e-05]]


box and soap

In [3]:
calcpos = np.mean(atomic_coordinates, axis=0)
Box = Atoms(
    cell=[
        [100.6402, 0.0, 0.0],
        [0.0, 100.6402, 0.0],
        [0.0, 0.0, 100.6402]
    ],
    positions = atomic_coordinates,
    
    symbols=atomic_symbols,
)


soaper = SOAP(
    r_cut=6,
    n_max=10,
    l_max=10,
    species=atomic_symbols1,
    rbf='gto',
    sparse=False
)

check

In [4]:
soap1 = soaper.create(Box, centers=[calcpos])
#print("Size of descriptor: {}\n".format(soap1.shape[1]))
#print("First five values, for position \n {}: \n{}".format(atomic_coordinates, soap1[0,:5]))
print(soap1[0,:5])

[0.00028533 0.00110428 0.00293385 0.00661465 0.01369122]


In [5]:
print(soap1.shape)

(1, 2310)


In [6]:
print(atomic_coordinates.shape)

(18, 3)


In [5]:
import os
import numpy as np
import pandas as pd
from ase import Atoms
from dscribe.descriptors import SOAP

# Function to calculate SOAP descriptors for a given XYZ file
def calculate_soap(xyz_file_path):
    with open(xyz_file_path, 'r') as xyz_file:
        lines = xyz_file.readlines()[2:]
    atomic_symbols = [line.split()[0] for line in lines]
    atomic_coordinates = np.array([line.split()[1:4] for line in lines], dtype=float)
    atomic_symbols1 = list(set(atomic_symbols))
    
    calcpos = np.mean(atomic_coordinates, axis=0)
    Box = Atoms(
        cell=[
            [100.6402, 0.0, 0.0],
            [0.0, 100.6402, 0.0],
            [0.0, 0.0, 100.6402]
        ],
        positions=atomic_coordinates,
        symbols=atomic_symbols,
    )
    
    # soaper = SOAP(
    #     r_cut=4,
    #     n_max=5,
    #     l_max=6,
    #     species=atomic_symbols1,
    #     rbf='gto',
    #     sparse=False
    # )
    # wersja minimalna
    soaper = SOAP(
    r_cut=3.0,        # Promień odcięcia
    n_max=3,          # Liczba funkcji radialnych
    l_max=2,          # Maksymalny moment kątowy
    species=atomic_symbols1,
    rbf='gto',        # Funkcja radialna
    sparse=False      # Nieskompresowany format
)
    
    soap_descriptor = soaper.create(Box, centers=[calcpos])
    return soap_descriptor

# Directory containing the subdirectories with XYZ files
base_dir = '../data/rop313/orop'

# List to store the results
results = []

# Iterate through the subdirectories
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    if os.path.isdir(subdir_path):
        xyz_file_path = os.path.join(subdir_path, '1.b973c.xyz')
        if os.path.exists(xyz_file_path):
            soap_descriptor = calculate_soap(xyz_file_path)
            results.append({
                'subdir': subdir,
                'soap_descriptor': soap_descriptor.tolist()
            })

# Convert results to a DataFrame and save to a CSV file
df_results = pd.DataFrame(results)
df_results.to_csv('../data/soap_descriptors_minimal.csv', index=False)

print("SOAP descriptors calculated and saved to soap_descriptors_minimal.csv")

SOAP descriptors calculated and saved to soap_descriptors_minimal.csv
